Este cuaderno explica cómo hacer un seguimiento del uso de tokens para llamadas específicas. Actualmente, esto solo se implementa para la API de OpenAI.

Comencemos con un ejemplo extremadamente simple de seguimiento del uso de tokens para una única llamada a un LLM.

In [1]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2) # type: ignore

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 39
	Prompt Tokens: 4
	Completion Tokens: 35
Successful Requests: 1
Total Cost (USD): $0.0007800000000000001


Todo lo que esté dentro del gestor de contexto será rastreado. Aquí tienes un ejemplo de cómo usarlo para hacer un seguimiento de múltiples llamadas en secuencia.

In [2]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    result2 = llm("Tell me a joke")
    print(cb.total_tokens)

81


If a chain or agent with multiple steps in it is used, it will track all those steps.

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.callbacks import get_openai_callback
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0) # type: ignore
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

with get_openai_callback() as cb:
    response = agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: "Olivia Wilde boyfriend"
Observation: Looks like Olivia Wilde and Jason Sudeikis are starting 2023 on good terms. Amid their highly publicized custody battle – and the actress' ...
Thought: I need to find out Jason Sudeikis' age
Action: Search
Action Input: "Jason Sudeikis age"
Observation: 47 years
Thought: I need to calculate 47 raised to the 0.23 power
Action: Calculator
Action Input: 47^0.23
Observation: Answer: 2.4242784855673896
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


 I now know the final answer
Final Answer: Jason Sudeikis is Olivia Wilde's boyfriend and his current age raised to the 0.23 power is 2.4242784855673896.

> Finished chain.
Total Tokens: 1526
Prompt Tokens: 1365
Completion Tokens: 161
Total Cost (USD): $0.030520000000000005
